In [2]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
import os
from torch.utils.data import Dataset,Subset
import pandas as pd
import re

In [3]:
file_path = '/media/saitomar/Work/Projects/DeepLense_Test/task_2_dataset/files/imageEUC_VIS-100000.fits'
hdul = fits.open(file_path)

In [4]:
class FitDataset(Dataset):
    def __init__(self, folder_path : str, label_file : str):
        
        self.folder_path = folder_path
        self.label_file = label_file
        self.filenames = os.listdir(self.folder_path)
        self.csv_file = pd.read_csv(self.label_file)

    def __len__(self):
        return len(self.filenames)
    
    def __getitem__(self, index):
        
        filename = os.path.join(self.folder_path, self.filenames[index])
        ID = re.findall('\d+',self.filenames[index])[0]
        
        df = self.csv_file.loc[self.csv_file['ID'] == int(ID)]
        label = df['is_lens'].values

        with fits.open(filename) as hdul:
            data = hdul[0].data

        data = data.astype(np.float32)
        tensor_data = torch.from_numpy(data).float().unsqueeze(dim=0)

        return tensor_data, label[0]
    


In [5]:
ds = FitDataset('/media/saitomar/Work/Projects/DeepLense_Test/task_2_dataset/files', '/media/saitomar/Work/Projects/DeepLense_Test/task_2_dataset/classifications.csv')


In [15]:
# rows = 4
# columns = 4

# fig, axes = plt.subplots(nrows=rows, ncols=columns, figsize=(columns*3, rows*3))
for i,(sample, label) in enumerate(ds):
    print(sample.shape,label)
    # fig.add_subplot(rows, columns, i+1)
    # plt.imshow(sample.squeeze(dim=0))
    # plt.xlabel(f'Label: {label}')

    if i == 16:
        break


torch.Size([1, 101, 101]) 1
torch.Size([1, 101, 101]) 1
torch.Size([1, 101, 101]) 1
torch.Size([1, 101, 101]) 1
torch.Size([1, 101, 101]) 0
torch.Size([1, 101, 101]) 0
torch.Size([1, 101, 101]) 0
torch.Size([1, 101, 101]) 1
torch.Size([1, 101, 101]) 1
torch.Size([1, 101, 101]) 0
torch.Size([1, 101, 101]) 1
torch.Size([1, 101, 101]) 1
torch.Size([1, 101, 101]) 0
torch.Size([1, 101, 101]) 1
torch.Size([1, 101, 101]) 0
torch.Size([1, 101, 101]) 0
torch.Size([1, 101, 101]) 0
